## Let's figure out how to section the data and massage it into whatever form we want to input it as

### Then we can paste the streamlined version into the full training notebooks

### Load Data and do 2d analysis

In [1]:
import sys
import numpy as np
from dataframework.src.datasets.vpicdataset import VPICDataset
from numpy.random import default_rng as generator
import h5py

In [2]:
def clear_dataset(group, key):
    try:  # if the dataset exists from previous failed attempts to do this, delete it
        del group[key]
        print(f"Overwrote existing dataset {group, key}")
    except:
        pass    

In [3]:
zooms = [[-np.inf,np.inf], [-40, 40]]
time_idx = 11 #the time index we are processing rn
smoothing = 3

In [4]:
files = ['/tigress/kendrab/03082021/data.h5','tigress/kendrab/03082021/info']
writepath = '/tigress/kendrab/03082021'
unique_seed = 308
kwargs = {'get_vars' : ['bx','by','bz','jy','vz']}
sim_dset = VPICDataset(vpicfiles=files, **kwargs)


NO PARAMS ADDED, FUNCTIONALIITY NOT ADDED YET!!!! SORRY
Added bx Variable
Added by Variable
Added bz Variable
Added jy Variable
Added vz Variable


In [5]:
# zoom in
desired_time = sim_dset.timeseries[time_idx]
dt = sim_dset.timeseries[1] - sim_dset.timeseries[0]
sim_dset = sim_dset.ndslice(timelims=[desired_time - dt/2, desired_time + dt/2], zooms=zooms)

In [6]:
# do the thing
sim_dset.find_structures(b1_name='bz', b2_name='bx')

Finding structures at simulation time 109.96344757080078
parameter d_per_de = 4
Added bz_smooth Variable
Added bx_smooth Variable
Added flux_fn Variable
Number of nulls:  25
parameter x_coords = [[  11.77526243  176.4494887 ]
 [  39.81400752  176.6009484 ]
 [  64.35528987  176.57593385]
 [  90.97669557  176.51127311]
 [4018.24230104  176.55062254]
 [4429.4040307   176.52519135]
 [4675.15171881  176.54090599]
 [4875.94837936  176.4614515 ]
 [5024.3470557   176.49916782]
 [8740.28029513  176.55462583]
 [8787.33698135  176.47466309]
 [8848.87110388  176.45629907]]
parameter o_coords = [[1.88023194e+01 1.76535905e+02]
 [8.43208467e+00 1.76444992e+02]
 [5.14821014e+01 1.76591613e+02]
 [6.95054837e+01 1.76557884e+02]
 [1.04670412e+02 1.76509984e+02]
 [4.16597877e+03 1.76470618e+02]
 [4.55914018e+03 1.76461529e+02]
 [4.76476044e+03 1.76500938e+02]
 [4.98691283e+03 1.76501496e+02]
 [8.73534397e+03 1.76570176e+02]
 [8.77032066e+03 1.76512916e+02]
 [8.83745797e+03 1.76489099e+02]
 [8.86662096e+0

### Change the coordinate system to match MMS magnetotail expectations

### Take random slices

In [7]:
samples = 1000
min_dist = 30
max_dist = 200
rand = generator(seed=unique_seed*time_idx)

def random_bounded(bounds):
    """ Use numpy to generate an array of random numbers with specified mins and maxes
    
    Parameters
    ----------
    bounds : array, shape (...,2)
        holds the bounds needed for each point in the random array. bounds[...,0] has mins
        
    Returns
    -------
    rands : array, shape (...)
        the array of random numbers returned
    """
    unit_rands = rand.random(size=bounds.shape[:-1])
    rands = (bounds[...,1] - bounds[...,0])*unit_rands + bounds[...,0]
    
    return rands

def mtail_transform(sim_data):
    """ The geometry of the simulation is different wrt the magnetotail. Keeping x the same, bx will be
    the mtail's 'bz' and bz will be the mtails 'bx'. For this the z and y components must be flipped 180
    THIS IS NOT A GOOD GENERALIZABLE FUNCTION!!!! REUSE WITH EXTREME CAUTION!!!
    Assumes origin centered etc.
    """
    # Flip z component of everybody
    for key in sim_data.variables.keys():
        sim_data.variables[key].data = np.flip(sim_data.variables[key].data, axis=-1)
    # Flip vector quantities
    sim_data.variables['bz'].data *= -1
    sim_data.variables['jy'].data *= -1
    sim_data.variables['vz'].data *= -1
    
    

In [8]:
bounds = sim_dset.bounds(time=False)
segment_bounds = np.expand_dims(bounds, axis=-2)
segment_bounds = np.concatenate([segment_bounds for i in range(2)], axis=-2)
varkwargs = {'separatricesinterp' : 'nearest', 'o_structuresinterp' : 'nearest',
            'current_sheetsinterp' : 'nearest'}  # to use nearest interpolation for topography info
endpts_list = []
# slices_dict format: slices_dict['variable'] = [[var_from_slice_1, var_from_slice_2, ...]]
# Also stores the parameters
slices_dict={}
# transform the data to a more magnetotail-esque config
mtail_transform(sim_dset)

for i in range(samples):
    for j in range(1000):
        endpts = np.array(random_bounded(segment_bounds))  # in [[minx, maxx], [miny, maxy]] format
        if not np.all(np.abs(endpts[:,0] - endpts[:,1]) > 1):
            continue
        if max_dist > np.linalg.norm(endpts[:,0] - endpts[:,1]) > min_dist:
            break
    else:
        raise Error("Suitable random 1d segment not found, check minimum and maximum distances.")    
    slice = sim_dset.ndslice(set_pts=endpts.T, zooms=endpts, **varkwargs)
    endpts_list.append(endpts.T)  # to make first dimension the number of points
    # sanity check variable to make sure the slices are happening correctly
    orig_mesh = np.stack([slice.params['zero_pt'] 
                          + slice.params['unit_vec'] * slice.default_mesh[0][i] 
                          for i in range(len(slice.default_mesh[0]))], axis=-1)
    for var in slice.variables.keys(): # this will lose info if the variables have different meshes
                                       #   so will need to interpolate all to the same mesh first to avoid
        slices_dict.setdefault(var,[]).append(slice.variables[var].data.reshape(-1))
    for param in slice.params.keys(): 
        slices_dict.setdefault(param,[]).append(slice.params[param])
    cs = slice.variables['current_sheets'].data[0]
    os = slice.variables['o_structures'].data[0]
    slices_dict.setdefault('s',[]).append(slice.default_mesh[0].reshape(-1))
    # neither 0
    # os 1
    # cs 2
    # both 3
    slices_dict.setdefault('topo',[]).append((2*cs+os).reshape(-1))


overriding interpolation type linear in variable separatrices with nearest
overriding interpolation type linear in variable o_structures with nearest
overriding interpolation type linear in variable current_sheets with nearest
overriding interpolation type linear in variable separatrices with nearest
overriding interpolation type linear in variable o_structures with nearest
overriding interpolation type linear in variable current_sheets with nearest
overriding interpolation type linear in variable separatrices with nearest
overriding interpolation type linear in variable o_structures with nearest
overriding interpolation type linear in variable current_sheets with nearest
overriding interpolation type linear in variable separatrices with nearest
overriding interpolation type linear in variable o_structures with nearest
overriding interpolation type linear in variable current_sheets with nearest
overriding interpolation type linear in variable separatrices with nearest
overriding interp

### Save the slice data to disk

In [9]:
writefile = h5py.File(writepath+"/"+str(samples)+"samples_idx"+str(time_idx)+'_' +''.join(kwargs['get_vars']) +'.hdf5', 'a')

for key in slices_dict.keys():  # this is a mess. Is this how you are supposed to use try except blocks? Prob not
    clear_dataset(writefile, key)
    
    if type(slices_dict[key]) is np.ndarray:
        dset = writefile.create_dataset(key, data=slices_dict[key])
        continue
        
    try:
        dtype = h5py.vlen_dtype(type(slices_dict[key][0].flatten()[0])) #vlen dtype is list of arrays
        print(key, type(dtype))
        dset = writefile.create_dataset(key, (len(slices_dict[key]),), dtype=dtype)
        for i in range(len(slices_dict[key])): #ew :(
            writefile[key][i] = slices_dict[key][i] 
              
    except:
        clear_dataset(writefile, key)  # clear failed attempt
        dt = h5py.special_dtype(vlen=str)
        dset = writefile.create_dataset(key, (len(slices_dict[key]),), dtype=dt)
        print(key, type(dtype))
        for i in range(len(slices_dict[key])): #ew :(
            writefile[key][i] = str(slices_dict[key][i])

bx <class 'numpy.dtype'>
by <class 'numpy.dtype'>
bz <class 'numpy.dtype'>
jy <class 'numpy.dtype'>
vz <class 'numpy.dtype'>
bz_smooth <class 'numpy.dtype'>
bx_smooth <class 'numpy.dtype'>
flux_fn <class 'numpy.dtype'>
fluxfn_hessian_det <class 'numpy.dtype'>
separatrices <class 'numpy.dtype'>
o_structures <class 'numpy.dtype'>
current_sheets <class 'numpy.dtype'>
d_per_de <class 'numpy.dtype'>
x_coords <class 'numpy.dtype'>
Overwrote existing dataset (<HDF5 file "1000samples_idx11_bxbybzjyvz.hdf5" (mode r+)>, 'x_coords')
x_coords <class 'numpy.dtype'>
o_coords <class 'numpy.dtype'>
Overwrote existing dataset (<HDF5 file "1000samples_idx11_bxbybzjyvz.hdf5" (mode r+)>, 'o_coords')
o_coords <class 'numpy.dtype'>
unit_vec <class 'numpy.dtype'>
zero_pt <class 'numpy.dtype'>
s <class 'numpy.dtype'>
topo <class 'numpy.dtype'>


close the output file

In [10]:
writefile.close()

### Write the 2d stuff to a separate file

In [11]:
writefile = h5py.File(writepath+"/"+"idx"+str(time_idx)+'_' +''.join(kwargs['get_vars']) +'.hdf5', 'a')

for key in sim_dset.variables.keys():  # this is a mess. Is this how you are supposed to use try except blocks? Prob not
    clear_dataset(writefile, key)
    dset = writefile.create_dataset(key, data=sim_dset.variables[key].data)

for key in sim_dset.params.keys():
    clear_dataset(writefile, key)
    dset = writefile.create_dataset(key, data=sim_dset.params[key])

close the output file

In [12]:
writefile.close()

### Visualize the slices (optional)

In [13]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

flux_fn = sim_dset.variables['flux_fn']

X,Y = np.meshgrid(*flux_fn.mesh, indexing='ij')
fig, ax = plt.subplots(figsize=(10,2))
ctr =ax.contour(X,Y,flux_fn.data[0], levels=400)
plt.colorbar(mappable=ctr)
paths = LineCollection(endpts_list, colors='black')
ax.add_collection(paths)

for endpts in endpts_list:
    print(endpts[0])
    ax.scatter(*endpts[0], color='black') # marks starting pt
    
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-906.98066728  -11.41164639]
[-35.76191778  -5.65663767]
[-76.2309278    6.89108561]
[-170.11935507    2.70991078]
[605.29851451  31.2734549 ]
[-262.06097505   34.51331801]
[-316.48113711  -33.57484233]
[-80.22366137  -9.99301976]
[-309.28029937   38.63698512]
[-419.98559953  -21.79813886]
[270.91314135  -9.78394016]
[-369.65682783   -6.81873671]
[-742.95191938   11.33079227]
[-330.70769064   29.28620536]
[-479.43676793    6.59588108]
[927.95481414 -31.45306102]
[207.63142658   9.24381525]
[-656.35243456   -5.71432359]
[-11.62355417  39.58735382]
[376.87013407 -35.99584292]
[269.56733897 -22.51494609]
[ 77.77374637 -28.61103703]
[-283.58227373   -6.36611645]
[-632.27646983  -10.39430842]
[922.60707173 -12.48441409]
[247.79592456  20.71294989]
[210.93515148 -23.37896644]
[139.60762772  15.2946514 ]
[-962.77559169  -21.10958573]
[178.57290753 -37.31544231]
[564.22444155   0.76057259]
[641.12423688  12.38995566]
[-706.21333203  -24.93182027]
[414.87725276 -25.70337217]
[-4.81534017e+02 -